In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
articles = pd.read_csv('../data/processed/articles_final_features.csv')

In [3]:
articles.head()

,Unnamed: 0,HEADLINE,HEADLINE_LEN,PUBLISH_DATE,DAY_OF_WEEK,PUBLISH_MONTH,PUBLISHTIMEINSECONDS,WEEKDAY,HOLIDAY,BUREAU,ARTICLE_TYPE,PROMO,CUM_SESSIONS_SOFAR,HEADLINE_GROUP,PROMO_GROUP,SP_INDEX,gtrends_bureau_index
0,0,2 under-the-radar tech stocks to buy in 2022,45,2022-04-12,1,4,22740,Weekday,No,technology-and-telecom,article,These two companies an enjoying explosive top-...,207,0,3,4397.45,18.0
1,1,does it matter that gamestop's split will be a...,63,2022-04-12,1,4,32400,Weekday,No,consumer-goods,article,Just how different is it from a cash dividend?,5811,2,1,4397.45,63.0
2,2,is amazon stock a buy this month?,34,2022-04-12,1,4,38220,Weekday,No,consumer-goods,article,There are strong reasons to invest in Amazon r...,1637,5,2,4397.45,63.0
3,3,2 stocks that cut you a check each month,41,2022-04-12,1,4,33420,Weekday,No,industrials,article,"For some investors, dividend income that flows...",2037,0,6,4397.45,0.0
4,4,is amazon or alphabet the better stock split i...,57,2022-04-12,1,4,62220,Weekday,No,technology-and-telecom,article,Both companies dominate much of our digital li...,7682,2,1,4397.45,18.0


In [4]:
articles.drop(columns='Unnamed: 0', inplace=True)

In [5]:
articles.head(5)

,HEADLINE,HEADLINE_LEN,PUBLISH_DATE,DAY_OF_WEEK,PUBLISH_MONTH,PUBLISHTIMEINSECONDS,WEEKDAY,HOLIDAY,BUREAU,ARTICLE_TYPE,PROMO,CUM_SESSIONS_SOFAR,HEADLINE_GROUP,PROMO_GROUP,SP_INDEX,gtrends_bureau_index
0,2 under-the-radar tech stocks to buy in 2022,45,2022-04-12,1,4,22740,Weekday,No,technology-and-telecom,article,These two companies an enjoying explosive top-...,207,0,3,4397.45,18.0
1,does it matter that gamestop's split will be a...,63,2022-04-12,1,4,32400,Weekday,No,consumer-goods,article,Just how different is it from a cash dividend?,5811,2,1,4397.45,63.0
2,is amazon stock a buy this month?,34,2022-04-12,1,4,38220,Weekday,No,consumer-goods,article,There are strong reasons to invest in Amazon r...,1637,5,2,4397.45,63.0
3,2 stocks that cut you a check each month,41,2022-04-12,1,4,33420,Weekday,No,industrials,article,"For some investors, dividend income that flows...",2037,0,6,4397.45,0.0
4,is amazon or alphabet the better stock split i...,57,2022-04-12,1,4,62220,Weekday,No,technology-and-telecom,article,Both companies dominate much of our digital li...,7682,2,1,4397.45,18.0


### What is the Baseline?

In [44]:
articles['CUM_SESSIONS_SOFAR'].mean()

7086.222303995844

In [72]:
articles['baseline'] = articles['CUM_SESSIONS_SOFAR'].mean()

In [75]:
articles['baseline_diff'] = articles['CUM_SESSIONS_SOFAR'] - articles['baseline'] 

In [77]:
np.mean(abs(articles['baseline_diff']))

7617.685621018552

### Set X & y

In [34]:
X = articles[[           
'HEADLINE_LEN',          
'DAY_OF_WEEK',         
'PUBLISH_MONTH',       
'PUBLISHTIMEINSECONDS',
'WEEKDAY',             
'HOLIDAY',             
'BUREAU',              
'ARTICLE_TYPE',                       
'HEADLINE_GROUP',      
'PROMO_GROUP',                       
'SP_INDEX',                       
'gtrends_bureau_index'                                             
                                   ]]

In [35]:
y = articles['CUM_SESSIONS_SOFAR']

### Train, Test, Split

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=X['BUREAU'])

### One Hot Encode Categorical Variables:
- WEEKDAY
- HOLIDAY
- BUREAU
- ARTICLE TYPE

In [38]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [39]:
# create encoder with a column transformer

smart_encoder = make_column_transformer(
    (OneHotEncoder(drop = 'if_binary'), ['WEEKDAY', 'HOLIDAY', 'BUREAU', 'ARTICLE_TYPE']), remainder='passthrough'
                       )                        
                        

In [40]:
# fit and transform on X train
X_train_ohe = smart_encoder.fit_transform(X_train)


In [41]:
X_train_ohe

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 6.00000e+00,
        4.41013e+03, 1.90000e+01],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        3.36047e+03, 1.80000e+01],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 5.00000e+00,
        3.53422e+03, 3.10000e+01],
       ...,
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        3.37285e+03, 4.20000e+01],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        3.44344e+03, 2.10000e+01],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        4.39794e+03, 4.20000e+01]])

In [42]:
# save as df
pd.DataFrame(X_train_ohe, columns=smart_encoder.get_feature_names())

,onehotencoder__x0_Weekend,onehotencoder__x1_Yes,onehotencoder__x2_cannabis,onehotencoder__x2_consumer-goods,onehotencoder__x2_cryptocurrency,onehotencoder__x2_energy-materials-and-utilities,onehotencoder__x2_financials,onehotencoder__x2_health-care,onehotencoder__x2_industrials,onehotencoder__x2_investment-planning,...,onehotencoder__x3_transcript,onehotencoder__x3_video,HEADLINE_LEN,DAY_OF_WEEK,PUBLISH_MONTH,PUBLISHTIMEINSECONDS,HEADLINE_GROUP,PROMO_GROUP,SP_INDEX,gtrends_bureau_index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,73.0,0.0,1.0,33300.0,0.0,6.0,4410.13,19.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,45.0,0.0,8.0,54780.0,2.0,1.0,3360.47,18.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,76.0,0.0,10.0,65640.0,3.0,5.0,3534.22,31.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,58.0,2.0,2.0,41340.0,3.0,3.0,4225.50,20.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,33.0,5.0,1.0,32100.0,3.0,1.0,4662.85,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,62.0,4.0,10.0,25200.0,3.0,1.0,3477.13,28.0
17320,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,53.0,3.0,2.0,5400.0,9.0,4.0,4504.08,44.0
17321,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,40.0,5.0,8.0,32400.0,0.0,1.0,3372.85,42.0
17322,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,60.0,2.0,11.0,48600.0,1.0,0.0,3443.44,21.0


In [43]:
# only transform X Test
X_test_ohe = smart_encoder.transform(X_test)

In [74]:
articles.head()

,HEADLINE,HEADLINE_LEN,PUBLISH_DATE,DAY_OF_WEEK,PUBLISH_MONTH,PUBLISHTIMEINSECONDS,WEEKDAY,HOLIDAY,BUREAU,ARTICLE_TYPE,PROMO,CUM_SESSIONS_SOFAR,HEADLINE_GROUP,PROMO_GROUP,SP_INDEX,gtrends_bureau_index,baseline
0,2 under-the-radar tech stocks to buy in 2022,45,2022-04-12,1,4,22740,Weekday,No,technology-and-telecom,article,These two companies an enjoying explosive top-...,207,0,3,4397.45,18.0,7086.222304
1,does it matter that gamestop's split will be a...,63,2022-04-12,1,4,32400,Weekday,No,consumer-goods,article,Just how different is it from a cash dividend?,5811,2,1,4397.45,63.0,7086.222304
2,is amazon stock a buy this month?,34,2022-04-12,1,4,38220,Weekday,No,consumer-goods,article,There are strong reasons to invest in Amazon r...,1637,5,2,4397.45,63.0,7086.222304
3,2 stocks that cut you a check each month,41,2022-04-12,1,4,33420,Weekday,No,industrials,article,"For some investors, dividend income that flows...",2037,0,6,4397.45,0.0,7086.222304
4,is amazon or alphabet the better stock split i...,57,2022-04-12,1,4,62220,Weekday,No,technology-and-telecom,article,Both companies dominate much of our digital li...,7682,2,1,4397.45,18.0,7086.222304


### Linear Regression

In [45]:
from sklearn.linear_model import LinearRegression

In [46]:
# instantiate
lr = LinearRegression()

In [52]:
# fit 
lr.fit(X_train_ohe, y_train)

LinearRegression()

### Feature Importance

In [54]:
lr.coef_  ### how to map these to the features themselves??

array([ 3.77167211e+03,  5.70546974e+03,  7.17299350e+02, -2.42567933e+03,
        3.67577552e+03, -8.43647130e+02, -9.31581043e+02,  1.50440825e+03,
        1.02297951e+03,  2.72421081e+03, -3.24124259e+03, -2.59339023e+03,
        3.90866885e+02,  5.16512480e+01,  3.19624270e+03, -2.76431052e+03,
        8.65234810e+03, -6.01450169e+03, -3.12142983e+03,  4.48869285e+01,
       -1.87599094e+02, -2.80846570e+02, -1.98126136e-02, -5.71298445e+01,
        2.18185919e+01, -5.35613953e+00,  4.66947673e+00])

In [55]:
lr.intercept_

27186.303331491297

In [47]:
from sklearn.model_selection import cross_val_score

In [71]:
cross_val_score(lr, X_train_ohe, y_train, scoring='neg_mean_squared_error')
# cv controls the amount of folds
# this is returning the MSE- ignore the negative. 

array([-1.70256327e+08, -1.73747147e+08, -1.90860082e+08, -1.76389425e+08,
       -1.62910615e+08])

In [69]:
np.mean(cross_val_score(lr, X_train_ohe, y_train, scoring='neg_mean_squared_error'))

-174832719.05332547

In [70]:
np.sqrt(abs(np.mean(cross_val_score(lr, X_train_ohe, y_train, scoring='neg_mean_squared_error'))))

13222.43241817955

In [ ]:
# yikes, off by 13k sessions on average! way worse than the baseline of 7k. 

In [57]:
preds = lr.predict(X_test_ohe)

In [58]:
preds

array([ 5551.03819275,  2725.80954924,  4773.08244803, ...,
       11831.0633931 ,  2012.9118786 , 12073.70851562])

In [ ]:
# feataure permutation importance- shuffles features and helps figure out which features matter 
# and which don't
# sklearn has a feature importance library
# partial dependence can also help you evaluate your features

In [60]:
X_test['actual_sessions'] = y_test

/var/folders/kz/klxfgxt54zg3ld3hw3501lf80000gp/T/ipykernel_2884/3265505906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['actual_sessions'] = y_test


In [62]:
X_test['predicted_Sessions'] = preds

/var/folders/kz/klxfgxt54zg3ld3hw3501lf80000gp/T/ipykernel_2884/3097092204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['predicted_Sessions'] = preds


In [63]:
X_test

,HEADLINE_LEN,DAY_OF_WEEK,PUBLISH_MONTH,PUBLISHTIMEINSECONDS,WEEKDAY,HOLIDAY,BUREAU,ARTICLE_TYPE,HEADLINE_GROUP,PROMO_GROUP,SP_INDEX,gtrends_bureau_index,actual_sessions,predicted_Sessions
8439,53,0,9,61140,Weekday,No,technology-and-telecom,news brief,3,5,3351.60,24.0,794,5551.038193
6777,58,3,8,68400,Weekday,No,health-care,transcript,7,0,3373.43,17.0,586,2725.809549
11191,47,4,9,59640,Weekday,No,cannabis,news brief,3,1,3340.97,6.0,5221,4773.082448
3157,51,2,11,21060,Weekday,No,health-care,article,0,1,3443.44,32.0,28030,12022.120272
6971,82,6,8,29220,Weekend,No,health-care,article,3,6,3508.01,21.0,9305,16656.252824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12266,49,1,10,60060,Weekday,No,industrials,10% promise series,2,1,3360.95,49.0,4702,8409.029323
6770,75,0,8,82800,Weekday,No,health-care,transcript,7,0,3360.47,17.0,109,3835.798547
14234,44,1,1,21600,Weekday,No,cryptocurrency,article,3,1,4356.45,38.0,3877,11831.063393
10708,39,2,9,36060,Weekday,No,consumer-goods,news brief,3,1,3398.96,51.0,597,2012.911879


### Next Steps
- look at feature importance?
- try to polynomial?
- try knn?
- eventually try a time series?